In [1]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 100)

In [2]:
df = pd.read_csv('for_PCA_dong_ver2.csv')
df['수거함_결핍률'] = 1 - (df['수거함_개수']/df['총_거주인구수'])
df.head()
df.columns

Index(['자치구', '행정동', '행정동_코드', '월평균_총생활인구수', '연령대0_평균', '연령대10_평균', '연령대20_평균',
       '연령대30_평균', '연령대40_평균', '연령대50_평균', '연령대60_평균', '연령대70이상_평균', '1인가구',
       '2인가구', '3인가구', '4인가구', '5인가구', '6인가구', '7인 이상 가구', '연령대_100',
       '연령대_0_거주인구수', '연령대_10_거주인구수', '연령대_20_거주인구수', '연령대_30_거주인구수',
       '연령대_40_거주인구수', '연령대_50_거주인구수', '연령대_60_거주인구수', '연령대_70_거주인구수',
       '연령대_80_거주인구수', '연령대_90_거주인구수', '유흥주점영업', '단란주점', '제과점영업', '집단급식소',
       '일반음식점', '휴게음식점', '연령대_2030_거주인구수_합', '연령대_2030_거주인구수_평균',
       '연령대_20_거주인구수_비율', '연령대_30_거주인구수_비율', '연령대_2030_거주인구수_비율', '1인가구_비율',
       '식품접객업', '주점', '주요경제활동인구_거주인구수', '주요경제활동인구_생활인구수', 'PET예측량', '수거함_개수',
       '총_거주인구수', '수거함개수_당_거주인구수', '수거함_결핍률'],
      dtype='object')

### 1. VIF로 다중공선성 체크
- VIF 10이 넘는 변수 제거
- 초기 컬럼 선택 : '월평균_총생활인구수', '1인가구',
       '2인가구', '3인가구', '4인가구', '5인가구','유흥주점영업', '단란주점', '제과점영업', '집단급식소',
       '일반음식점', '휴게음식점', '연령대_2030_거주인구수_합', '연령대_2030_거주인구수_평균',
       '연령대_20_거주인구수_비율', '연령대_30_거주인구수_비율', '연령대_2030_거주인구수_비율', '1인가구_비율',
       '식품접객업', '주점', '주요경제활동인구_거주인구수', '주요경제활동인구_생활인구수', 'PET예측량', '수거함_개수',
       '총_거주인구수', '거주인구_당_수거함개수'

- 다중공선성 10 초과 컬럼 필터링 결과 : ['1인가구_비율','식품접객업', '주점', '주요경제활동인구_생활인구수', '수거함_개수','거주인구_당_수거함개수']
- 참고 : 상수항 (intercept)의 다중공선성이 높을 경우, 일반적으로 큰 문제가 되지 않음

In [3]:
x_features = ['1인가구_비율','식품접객업','주점','연령대_2030_거주인구수_합', '주요경제활동인구_생활인구수', '수거함개수_당_거주인구수'] #ㄷ, '월평균_총생활인구수', '수거함_결핍률', '주점']

# x_features에 대한 VIF 계산을 위해 상수(intercept) 열을 추가
x_features_with_const = sm.add_constant(df[x_features])

# VIF 계산
vif_data = pd.DataFrame()
vif_data["Variable"] = x_features_with_const.columns
vif_data["VIF"] = [variance_inflation_factor(x_features_with_const.values, i) for i in range(x_features_with_const.shape[1])]
vif_data.T

,0,1,2,3,4,5,6
Variable,const,1인가구_비율,식품접객업,주점,연령대_2030_거주인구수_합,주요경제활동인구_생활인구수,수거함개수_당_거주인구수
VIF,84.320209,4.933838,3.117784,1.939024,6.401678,2.297418,4.119225


### 2. Best Subset Selection
- 모든 가능한 독립 변수의 조합을 고려하여 최적의 모델을 선택하는 방법 
- 본 프로젝트에서는 AIC 를 채택
- AIC (Akaike Information Criterion):
AIC는 모델의 적합도와 모델의 복잡성을 고려하여 모델 선택을 합니다. <br>
AIC가 낮을수록 모델의 예측 성능이 좋으며, 데이터에 더 잘 적합됩니다.<br>
데이터에 대한 적합성을 높이는 데 초점을 맞추고, 모델의 복잡성을 상대적으로 덜 고려할 때 사용됩니다.<br>
**작은 크기의 데이터셋 또는 더 유연한 모델 선택**을 할 때 유용합니다.


- BIC (Bayesian Information Criterion):
BIC는 AIC와 마찬가지로 적합도와 복잡성을 고려하지만, 모델의 크기에 민감하게 반응합니다.<br>
BIC가 낮을수록 모델의 예측 성능이 좋으며, 데이터에 더 잘 적합됩니다. 그러나 BIC는 모델의 크기를 더 강력하게 제한합니다.<br>
데이터에 대한 적합성을 고려하면서, 불필요한 변수를 제한하고 모델의 단순성을 강조할 때 사용됩니다.<br>
**큰 크기의 데이터셋 또는 변수 선택에 제한**을 두고 싶을 때 유용합니다.

In [4]:
import itertools
import statsmodels.api as sm
import numpy as np

# 종속 변수 설정
y = df['PET예측량']

# 독립 변수 리스트 (const 를 제외한 변수들)
independent_variables = vif_data.iloc[1:, 0].tolist()

# 최적 모델 초기화
best_model = None
best_model_summary = None
best_aic = float('inf')

best_bic_model = None
best_bic_model_summary = None
best_bic = float('inf')

# Best Subset Selection
for k in range(1, len(independent_variables) + 1):
    for subset in itertools.combinations(independent_variables, k):
        X_subset = df[list(subset)]
        X_subset = sm.add_constant(X_subset)
        model = sm.OLS(y, X_subset).fit()
        
        aic = model.aic
        bic = model.bic
        
        if aic < best_aic:
            best_aic = aic
            best_aic_model = model
            best_aic_model_summary = model.summary()
        
        if bic < best_bic:
            best_bic = bic
            best_bic_model = model
            best_bic_model_summary = model.summary()

print("### Best Subset Selection 결과 (AIC):")
print(best_aic_model_summary)

# print('\n\n')

# print("### Best Subset Selection 결과 (BIC):")
# print(best_bic_model_summary)

### Best Subset Selection 결과 (AIC):
                            OLS Regression Results                            
Dep. Variable:                 PET예측량   R-squared:                       0.982
Model:                            OLS   Adj. R-squared:                  0.981
Method:                 Least Squares   F-statistic:                     1086.
Date:                Fri, 15 Dec 2023   Prob (F-statistic):           7.77e-69
Time:                        22:39:51   Log-Likelihood:                -267.42
No. Observations:                  85   AIC:                             544.8
Df Residuals:                      80   BIC:                             557.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
cons

### 3. 최종변수 + 추가변수 >> minmaxScaling >> PCA
- 추가변수 : (1 - 수거함수 / 거주인구수) = 수거함 결핍률
- 주성분 분석은 변수 간의 선형 결합으로 주성분을 구성

In [50]:
# df['수거함_결핍률'] = 1 - (df['수거함_개수']/df['총_거주인구수'])

In [5]:
# pca 수행
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

for_pca = ['1인가구_비율', '연령대_2030_거주인구수_합', '주요경제활동인구_생활인구수', '수거함개수_당_거주인구수']
pre_pca = df[for_pca]

# 스케일링
scaler = MinMaxScaler()
pre_pca = scaler.fit_transform(pre_pca)
pre_pca = pd.DataFrame(pre_pca)

# PCA 모델 생성
pca = PCA()

# 데이터를 PCA 모델에 fitting
pca.fit(pre_pca)

# 개별 주성분의 설명 분산 비율 출력
explained_variance_ratio = pca.explained_variance_ratio_
print("## 개별 주성분의 설명 분산 비율:\n", explained_variance_ratio)

# 누적 분산 비율 출력
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)
print("## 누적 분산 비율:\n", cumulative_variance_ratio, '\n')

# 주성분(PC) 확인
principal_components = pca.components_

# 주성분의 Loadings 확인
loadings = pca.explained_variance_ratio_

# 주성분과 Loadings 출력
for i, (pc, loading) in enumerate(zip(principal_components, loadings), 1):
    print(f'주성분 {i}: {pc} (Explained Variance: {loading:.2f})')

## 개별 주성분의 설명 분산 비율:
 [0.51588353 0.329673   0.13556851 0.01887496]
## 누적 분산 비율:
 [0.51588353 0.84555653 0.98112504 1.        ] 

주성분 1: [0.45799349 0.60474072 0.49818966 0.41992581] (Explained Variance: 0.52)
주성분 2: [-0.71247587 -0.05224458  0.14062013  0.68547401] (Explained Variance: 0.33)
주성분 3: [-0.12663909 -0.37521307  0.85473497 -0.33556792] (Explained Variance: 0.14)
주성분 4: [ 0.51631642 -0.70055287 -0.03822523  0.4911027 ] (Explained Variance: 0.02)


#### 주성분 1 loading 채택(소수점 3째자리에서 반올림, 절댓값)

### 4. 행정동 별 지표 산출
- 전 변수 minmaxScaling
- 지표 계산

In [8]:
# 행정동 데이터 MinMax스케일링
fin_features = ['자치구', '행정동', '행정동_코드', '1인가구_비율', '연령대_2030_거주인구수_합', '주요경제활동인구_생활인구수', '수거함개수_당_거주인구수']
df = df[fin_features]

scaler = MinMaxScaler()
df.iloc[:, 3:] = scaler.fit_transform(df.iloc[:, 3:])
df['SCORE'] = df['1인가구_비율']*0.46 + df['연령대_2030_거주인구수_합']*0.60 + df['주요경제활동인구_생활인구수']*0.50 +  df['수거함개수_당_거주인구수']*0.42 
df['SCORE'] = np.round(df['SCORE'], 2)
# df.head()
df_ = df.sort_values(by='SCORE', ascending = False).head()

In [9]:
df['최종_스코어'] = df['SCORE'] /  1.510000 * 100
df.sort_values(by='최종_스코어', ascending = False).head()

,자치구,행정동,행정동_코드,1인가구_비율,연령대_2030_거주인구수_합,주요경제활동인구_생활인구수,수거함개수_당_거주인구수,SCORE,최종_스코어
41,강서구,화곡1동,11500540,0.581286,1.000000,0.573449,0.849013,1.51,100.000000
60,관악구,청룡동,11620595,0.988142,0.878827,0.468999,0.245140,1.32,87.417219
51,영등포구,영등포동,11560535,1.000000,0.600803,0.745670,0.010561,1.20,79.470199
28,은평구,진관동,11380690,0.258060,0.526623,0.670115,1.000000,1.19,78.807947
55,동작구,상도1동,11590530,0.594856,0.686916,0.472196,0.574032,1.16,76.821192


In [10]:
df.to_csv('행정동_지표포함.csv', encoding = 'utf-8-sig', index = False)